In [3]:
import os
import numpy as np
import mne
import mne_bids


def mov_av(x):
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return np.hstack((x[:2], (cumsum[5:] - cumsum[:-5]) / 5.0, x[-2:]))


def trans_to_optimal_position(runs, subject, task, bids_derivatives, bids_root, head_pos):

    os.environ['OMP_NUM_THREADS'] = '1'

    displacement_threshold = 10      # mm
    velocity_threshold = 0.04        # m/s
    rotation_threshold = 10          # deg/s

    head_pos[:, 0] = head_pos[:, 0] - head_pos[0, 0]
    times = head_pos[:, 0]
    times_diff = np.diff(times)

    deg = mne.transforms.quat_to_rot(head_pos[:, 1:4])
    rotation = np.vstack((deg[:, 2, 1], deg[:, 2, 0], deg[:, 1, 0])).T * 180 / np.pi

    rotation = np.vstack((mov_av(rotation[:, 0]),
                          mov_av(rotation[:, 1]),
                          mov_av(rotation[:, 2]))).T

    rotation_vel = np.diff(rotation, axis=0) / times_diff[:, np.newaxis]
    rotation_art_point = (np.abs(rotation_vel) > rotation_threshold).sum(axis=1).astype(bool)
    rotation_annot = times[:-1][rotation_art_point]

    velocity = np.diff(head_pos[:, 4:7], axis=0) / times_diff[:, np.newaxis]
    velocity = np.sqrt((velocity ** 2).sum(axis=1))

    velocity_art_point = velocity > velocity_threshold
    velocity_annot = times[:-1][velocity_art_point]

    movement_onset = np.concatenate((velocity_annot, rotation_annot))
    movement_annot = mne.Annotations(
        onset=movement_onset,
        duration=0.2,
        description=['bad_velocity'] * len(velocity_annot) + ['bad_rotation'] * len(rotation_annot)
    )

    good_points_index = (velocity_art_point + rotation_art_point - 1).astype(bool)

    optimal_points_index = []

    for x in range(len(head_pos) - 1):

        if not good_points_index[x]:
            optimal_points_index.append(len(head_pos))
            continue

        displacement = (head_pos[:, 4:7] - head_pos[x, 4:7]) * 1000
        disp_mag = np.sqrt((displacement[:, :3] ** 2).sum(axis=1))

        optimal_points_index.append((disp_mag[:-1][good_points_index] > displacement_threshold).sum())

    optimal_points_index = np.argmin(optimal_points_index)
    optimal_head_pos = head_pos[optimal_points_index, 4:7] * 1000

    displacement = (head_pos[:, 4:7] - head_pos[optimal_points_index, 4:7]) * 1000
    disp_mag = np.sqrt((displacement[:, :3] ** 2).sum(axis=1))
    disp_times = times[disp_mag > displacement_threshold]

    displacement_annot = mne.Annotations(
        onset=disp_times,
        duration=np.ones(len(disp_times)),
        description=['bad_displacement'] * len(disp_times)
    )

    bp_annot = mne_bids.BIDSPath(
        subject=subject,
        root=bids_derivatives,
        task=task,
        processing='tsss+mc+transtooptimal',
        extension='.tsv'
    )

    np.save(str(bp_annot.fpath) + '_optimalheadpos', optimal_head_pos)
    (movement_annot + displacement_annot).save(str(bp_annot.fpath) + '_annotations.txt', overwrite=True)

    
    for run in runs:

        bp_raw = mne_bids.BIDSPath(
            subject=subject,
            root=bids_root,
            task=task,
            run=run,
            datatype='meg',
            extension='.fif'
        )

        bp_out = mne_bids.BIDSPath(
            subject=subject,
            root=bids_derivatives,
            task=task,
            run=run,
            datatype='meg',
            processing='mc+transtooptimal',
            extension='.fif'
        )

        raw = mne.io.read_raw_fif(bp_raw.fpath, preload=True)

        # Creating HEAD transform
        trans_matrix = np.eye(4)
        trans_matrix[:3, 3] = optimal_head_pos / 1000.0

        trans = mne.transforms.Transform('meg', 'head', trans_matrix)
        raw.info['dev_head_t'] = trans

        #.fif extension
        out_path = str(bp_out.fpath)
        if not out_path.endswith('.fif'):
            out_path += '.fif'

        raw.save(out_path, overwrite=True)

In [31]:
subjects_names = ['Z201']

bids_root = r'D:\\ds005234\\'
bids_derivatives = bids_root + r'derivatives\\'
task = 'vowels'

for subject in subjects_names:

    raw_bids_path = mne_bids.BIDSPath(
        subject=subject,
        root=bids_root,
        task=task,
        datatype='meg'
    )

    runs = mne_bids.get_entity_vals(str(raw_bids_path.directory), 'run')

    pos_file = os.path.join(
        bids_derivatives,
        f"sub-{subject}",
        f"sub-{subject}_task-{task}.pos"
    )

    head_pos = mne.chpi.read_head_pos(pos_file)

    out_dir = os.path.join(bids_derivatives, f"sub-{subject}", "meg")
    os.makedirs(out_dir, exist_ok=True)

    trans_to_optimal_position(runs, subject, task, bids_derivatives, bids_root, head_pos)


Overwriting existing file.
Opening raw data file D:\ds005234\sub-Z201\meg\sub-Z201_task-vowels_run-01_proc-tsss+mc+transtooptimal_meg.fif...
    Range : 45000 ... 582999 =     45.000 ...   582.999 secs
Ready.
Reading 0 ... 537999  =      0.000 ...   537.999 secs...


C:\Users\Prateek Patel\AppData\Local\Temp\ipykernel_25996\3340046086.py:111: RuntimeWarning: Unknown processing history item 220
  raw = mne.io.read_raw_fif(bp_raw.fpath, preload=True)
C:\Users\Prateek Patel\AppData\Local\Temp\ipykernel_25996\3340046086.py:111: RuntimeWarning: Unknown processing history item 220
  raw = mne.io.read_raw_fif(bp_raw.fpath, preload=True)


Writing D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_run-01_proc-mc+transtooptimal.fif


C:\Users\Prateek Patel\AppData\Local\Temp\ipykernel_25996\3340046086.py:125: RuntimeWarning: This filename (D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_run-01_proc-mc+transtooptimal.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(out_path, overwrite=True)


Closing D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_run-01_proc-mc+transtooptimal.fif
[done]
Opening raw data file D:\ds005234\sub-Z201\meg\sub-Z201_task-vowels_run-02_proc-tsss+mc+transtooptimal_meg.fif...
    Range : 24000 ... 564999 =     24.000 ...   564.999 secs
Ready.
Reading 0 ... 540999  =      0.000 ...   540.999 secs...


...

Closing D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_run-02_proc-mc+transtooptimal.fif
[done]
Opening raw data file D:\ds005234\sub-Z201\meg\sub-Z201_task-vowels_run-03_proc-tsss+mc+transtooptimal_meg.fif...
    Range : 37000 ... 579999 =     37.000 ...   579.999 secs
Ready.
Reading 0 ... 542999  =      0.000 ...   542.999 secs...


C:\Users\Prateek Patel\AppData\Local\Temp\ipykernel_25996\3340046086.py:111: RuntimeWarning: Unknown processing history item 220
  raw = mne.io.read_raw_fif(bp_raw.fpath, preload=True)
C:\Users\Prateek Patel\AppData\Local\Temp\ipykernel_25996\3340046086.py:111: RuntimeWarning: Unknown processing history item 220
  raw = mne.io.read_raw_fif(bp_raw.fpath, preload=True)


Writing D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_run-03_proc-mc+transtooptimal.fif


C:\Users\Prateek Patel\AppData\Local\Temp\ipykernel_25996\3340046086.py:125: RuntimeWarning: This filename (D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_run-03_proc-mc+transtooptimal.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(out_path, overwrite=True)


Closing D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_run-03_proc-mc+transtooptimal.fif
[done]
